In [32]:
import pandas as pd
import sqlalchemy
import requests
from bs4 import BeautifulSoup
from splinter import Browser
import time
import pymongo

executable_path = {'executable_path': 'chromedriver.exe'}
#browser = Browser('chrome', **executable_path, headless=False)

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.mlbDB

In [33]:
get_url = 'https://www.mlb.com/'
html = requests.get(get_url)
soup = BeautifulSoup(html.text, 'html.parser')
#soup.prettify()

# top headlines on mlb.com

In [34]:
top_headlines = []
results = soup.select('div.l-grid div.p-headline-stack')
#there are three div.p-headline-stack s, but index 0 and 2 are hidden ones that don't contain the same articles.
#index 1 is for xs and sm, index 3 is for md, lg, xl and I used index 3. code works the same with index 1.
for result in results[3].select('li.p-headline-stack__headline'):
    if result.a['href'][:24] == 'https://www.mlb.com/news':
        #some news articles link to mlb.com/cut4 which has a completely different layout for the next part
        #so i just dropped those articles
        top_headlines.append({'headline':result.text, 'headline_url':result.a['href']})
print(top_headlines)

[{'headline': 'Chavis delivers walk-off hit for Red Sox in 10th', 'headline_url': 'https://www.mlb.com/news/red-sox-walk-it-off-against-rockies'}, {'headline': "JV tames familiar foe in Astros' 8th straight win", 'headline_url': 'https://www.mlb.com/news/justin-verlander-astros-win-eighth-straight'}, {'headline': "Puig, Reds stun Cubs for 1st walk-off win of '19", 'headline_url': 'https://www.mlb.com/news/reds-defeat-cubs-on-yasiel-s-puig-walk-off-hit'}, {'headline': 'Story exits game in Boston after collision', 'headline_url': 'https://www.mlb.com/news/trevor-story-exits-game-after-collision'}, {'headline': 'Mondesi goes above and beyond to turn DP', 'headline_url': 'https://www.mlb.com/news/adalberto-mondesi-turns-great-double-play'}, {'headline': 'Andujar to have season-ending shoulder surgery', 'headline_url': 'https://www.mlb.com/news/miguel-andujar-to-have-season-ending-surgery'}, {'headline': 'Braves prospect Riley homers in 2nd MLB AB', 'headline_url': 'https://www.mlb.com/news

# parse top headline articles and pull player names and IDs and add to prior dictionary

In [35]:
browser = Browser('chrome', **executable_path, headless=False)
#i = 0
for i, each_article in enumerate(top_headlines):
    get_url = each_article['headline_url']
#     html = requests.get(get_url)
#     soup = BeautifulSoup(html.text, 'html.parser')
# had to use chromedriver because for some articles, the player names were linked in <a href> but others,
# the request would get <forge-entity> that would be converted to <a> after it had already 'requests.get'-ed
    
    browser.visit(get_url)

    while True:
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')
        try:
            results = soup.select('div.template-article__content div.article-item__body')
            break
        except:
            time.sleep(0.5)
    
    article_info = []
    #within the body of the first (index=0) article (because this page will load multiple articles)
    articlebody_results = results[0].select('p a')
    #find any link
    for each_link in articlebody_results:
        #and if the article links to a player page...
        if each_link['href'][:27] == 'https://www.mlb.com/player/':
            #add their name and their player id (last 6 characters of url)
            #seems mlb.com content writers are good about listing full name when they link a player
            #so going to the page and pulling their name or using the API to pull their name is unnecessary
            #some articles don't properly link a player to their player page on first mention, but oh well.
            article_info.append({'player_name':each_link.text, 'player_id':each_link['href'][-6:]})
    #print(article_info)
    top_headlines[i]['players_in_article'] = article_info
    #needed a counter, i, to correctly add the name-list to the correct article
    #i += 1
browser.quit()
print(top_headlines)

[{'headline': 'Chavis delivers walk-off hit for Red Sox in 10th', 'headline_url': 'https://www.mlb.com/news/red-sox-walk-it-off-against-rockies', 'players_in_article': [{'player_name': 'Michael Chavis', 'player_id': '656308'}]}, {'headline': "JV tames familiar foe in Astros' 8th straight win", 'headline_url': 'https://www.mlb.com/news/justin-verlander-astros-win-eighth-straight', 'players_in_article': [{'player_name': 'Justin Verlander', 'player_id': '434378'}]}, {'headline': "Puig, Reds stun Cubs for 1st walk-off win of '19", 'headline_url': 'https://www.mlb.com/news/reds-defeat-cubs-on-yasiel-s-puig-walk-off-hit', 'players_in_article': [{'player_name': 'Yasiel Puig', 'player_id': '624577'}]}, {'headline': 'Story exits game in Boston after collision', 'headline_url': 'https://www.mlb.com/news/trevor-story-exits-game-after-collision', 'players_in_article': [{'player_name': 'Trevor Story', 'player_id': '596115'}, {'player_name': 'Raimel Tapia', 'player_id': '606132'}, {'player_name': 'P

In [36]:
db.top_headlines.drop() #drop since i want it fresh each time it's added to db

In [37]:
# for each_headline in top_headlines:
#     db.top_headlines.insert_one(each_headline)
db.top_headlines.insert_many(top_headlines)
#used a for loop for insert_one, but insert_many works on a list of dictionaries, so why not just use that.

# top most rockies headline, URL, referenced player names and IDs

In [78]:
get_url = 'https://www.mlb.com/rockies'
html = requests.get(get_url)
soup = BeautifulSoup(html.text, 'html.parser')
#top_cr_headline = []
results = soup.select('div.l-grid div.p-headline-stack')
top_result = results[3].select('li.p-headline-stack__headline')[0]
top_cr_headline = {'headline':top_result.text.strip(), 'headline_url':top_result.a['href']}
print(top_cr_headline)

{'headline': 'Iannetta, Oberg spend rare off-day at home', 'headline_url': 'https://www.mlb.com/rockies/news/chris-iannetta-scott-oberg-off-days-at-home'}


In [79]:
browser = Browser('chrome', **executable_path, headless=False)
get_url = top_cr_headline['headline_url']
#     html = requests.get(get_url)
#     soup = BeautifulSoup(html.text, 'html.parser')
# had to use chromedriver because for some articles, the player names were linked in <a href> but others,
# the request would get <forge-entity> that would be converted to <a> after it had already requests.get-ed

browser.visit(get_url)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

article_info = []
results = soup.select('div.template-article__content div.article-item__body')
articlebody_results = results[0].select('p a')
for each_name in articlebody_results:
    if each_name['href'][:27] == 'https://www.mlb.com/player/':
        article_info.append({'player_name':each_name.text, 'player_id':each_name['href'][-6:]})
top_cr_headline['players_in_article'] = article_info
browser.quit()
print(top_cr_headline)

{'headline': 'Iannetta, Oberg spend rare off-day at home', 'headline_url': 'https://www.mlb.com/rockies/news/chris-iannetta-scott-oberg-off-days-at-home', 'players_in_article': [{'player_name': 'Chris Iannetta', 'player_id': '455104'}, {'player_name': 'Scott Oberg', 'player_id': '623184'}]}


In [80]:
db.rockies.drop()

In [81]:
db.rockies.insert_one(top_cr_headline)

# most recent rockies video

In [10]:
cr_team_id = 115
video_search_url = 'https://www.mlb.com/video/search/tag/teamid-'+str(cr_team_id)
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(video_search_url)
html = browser.html
soup = BeautifulSoup(html,'html.parser')

results = soup.select('div.video-preview')
video_page_url = 'https://www.mlb.com' + results[0].select('a')[0]['href']
browser.visit(video_page_url)
time.sleep(5)
html = browser.html
soup2 = BeautifulSoup(html,'html.parser')
video_result = {}
video_wrapper = soup2.select('div#video-section__wrapper meta')
for each_meta in video_wrapper:
    if each_meta['itemprop'] == 'name':
        video_result['name'] = each_meta['content']
    elif each_meta['itemprop'] == "contentURL":
        video_result['url'] = each_meta['content']
    elif each_meta['itemprop'] == "description":
        video_result['desc'] = each_meta['content']
print(video_result)
browser.quit()

In [13]:
cr_team_id = 115
video_search_url = 'https://www.mlb.com/video/search/tag/teamid-'+str(cr_team_id)
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(video_search_url)

while True:
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    try:
        results = soup.select('div.video-preview')
        break
    except:
        time.sleep(0.5)

video_page_url = 'https://www.mlb.com' + results[0].select('a')[0]['href']
browser.visit(video_page_url)

while True:
    html = browser.html
    soup2 = BeautifulSoup(html,'html.parser')
    try:
        video_wrapper = soup2.select('div#video-section__wrapper meta')
        break
    except:
        time.sleep(0.5)

video_result = {}

for each_meta in video_wrapper:
    if each_meta['itemprop'] == 'name':
        video_result['name'] = each_meta['content']
    elif each_meta['itemprop'] == "contentURL":
        video_result['url'] = each_meta['content']
    elif each_meta['itemprop'] == "description":
        video_result['desc'] = each_meta['content']
print(video_result)
browser.quit()

{'name': "Murphy's game-tying RBI groundout", 'desc': 'Daniel Murphy grounds out to second, but is able to score Charlie Blackmon to tie the game at 5 in the top of the 7th inning', 'url': 'https://cuts.diamond.mlb.com/FORGE/2019/2019-05/15/4a0e7f66-07b7954a-7af88a56-csvm-diamondx64-asset_1280x720_59_4000K.mp4'}


In [12]:
db.rockies.insert_one(video_result)

# most recent video featuring hitting and/or pitching leader using ID?

In [65]:
# top_p = db.top_players.find()
# for _ in top_p:
#     print(str(int(_['player_id'])))

518516
641355
488726
466320
500874
643446


In [14]:
top_p = db.top_players.find()

browser = Browser('chrome', **executable_path, headless=False)

for each_top_p in top_p:
    player_id = str(int(each_top_p['player_id']))
    #all player_ids should be integers, but convert to int to str should fix any issues with
    #player_id being stored as a double or a string.
    video_search_url = 'https://www.mlb.com/video/search/tag/playerid-'+player_id

    browser.visit(video_search_url)

    while True:
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')
        try:
            results = soup.select('div.video-preview')
            break
        except:
            time.sleep(0.5)

    video_page_url = 'https://www.mlb.com' + results[0].select('a')[0]['href']

    browser.visit(video_page_url)
    
    while True:
        html = browser.html
        soup2 = BeautifulSoup(html,'html.parser')
        try:
            video_wrapper = soup2.select('div#video-section__wrapper meta')
            break
        except:
            time.sleep(0.5)
            
    video_result = {}
    
    for each_meta in video_wrapper:
        if each_meta['itemprop'] == 'name':
            video_result['name'] = each_meta['content']
        elif each_meta['itemprop'] == "contentURL":
            video_result['url'] = each_meta['content']
        elif each_meta['itemprop'] == "description":
            video_result['desc'] = each_meta['content']
    print(video_result)
    db.top_players.update_one({'player_id':each_top_p['player_id']},{'$set':video_result})
browser.quit()

{'name': 'Could Bumgarner be traded?', 'desc': "The guys on MLB Now discuss Madison Bumgarner's no-trade list", 'url': 'https://cuts.diamond.mlb.com/FORGE/2019/2019-05/13/c651f26e-4b1c5fa1-ea3a17aa-csvm-diamondx64-asset_1280x720_59_4000K.mp4'}
{'name': 'Recap: LAD 6, SD 3', 'desc': "Daily Recap: Joc Pederson cranked his 100th home run and Cody Bellinger added three RBIs, including a two-run jack in the Dodgers' 6-3 win", 'url': 'https://cuts.diamond.mlb.com/FORGE/2019/2019-05/14/36b7f6e7-9d7b9c79-1749a526-csvm-diamondx64-asset_1280x720_59_4000K.mp4'}
{'name': "Brantley's 2-run single", 'desc': "Michael Brantley hits a grounder past Josh Harrison and two runs score, increasing the Astros' lead to 5-0 in the top of the 2nd", 'url': 'https://cuts.diamond.mlb.com/FORGE/2019/2019-05/14/c2059bca-253be60a-9bce3f72-csvm-diamondx64-asset_1280x720_59_4000K.mp4'}
{'name': "Cabrera's run-scoring single", 'desc': "Melky Cabrera lines an RBI single to center field to score Starling Marte and cut the

# pull standings from mlb.com/standings

In [8]:
browser = Browser('chrome', **executable_path, headless=False)
browser.visit('https://www.mlb.com/standings')

#wait for tables to load
while True:
    html = browser.html
    try:
        tables_list = pd.read_html(html)
        break
    except:
        print('sleepy')
        time.sleep(0.5)

browser.quit()
#print(test)

sleepy


In [9]:
all_divisions = []
for i in range(0,11,2):
    all_divisions.append(tables_list[i])
print(all_divisions)

[  American League EastAL East   W   L    PCT    GB  WCGB  L10 STRK   RS   RA  \
0               E-Tampa BayTB  26  15  0.634     -     -  6-4   W2  185  123   
1             E-NY YankeesNYY  26  16  0.619   0.5  +3.5  8-2   W3  208  168   
2                 E-BostonBOS  22  20  0.524   4.5   0.5  8-2   L1  224  198   
3                E-TorontoTOR  17  25  0.405   9.5   5.5  2-8   L1  151  182   
4              E-BaltimoreBAL  14  28  0.333  12.5   8.5  3-7   L2  163  242   

   DIFF  X-W/L   HOME   AWAY >.500     Next Game  
0    62  28-13  12-10   14-5   8-7  May 17 @ NYY  
1    40  25-17   15-9   11-7   6-7  May 17 vs TB  
2    26  23-19   11-8  11-12   4-6   Live vs COL  
3   -31  17-25   8-13   9-12  5-11  May 16 @ CWS  
4   -79  14-28   6-15   8-13  7-19  May 16 @ CLE  ,   American League CentralAL Central   W   L    PCT    GB WCGB  L10 STRK   RS  \
0                    C-MinnesotaMIN  27  15  0.643     -    -  7-3   W2  222   
1                    C-ClevelandCLE  22  19  0.537 

In [15]:
# al_e_df = pd.read_html(html)[0]
# al_c_df = pd.read_html(html)[2]
# al_w_df = pd.read_html(html)[4]
# nl_e_df = pd.read_html(html)[6]
# nl_c_df = pd.read_html(html)[8]
# nl_w_df = pd.read_html(html)[10]

In [10]:
#all_divisions = [al_e_df,al_c_df,al_w_df,nl_e_df,nl_c_df,nl_w_df]

for each_div in all_divisions:
    each_div.rename(columns={'>.500':'over500'},inplace=True)

#need to rename column name ">.500" as it breaks later due to the period I think?

In [11]:
all_divisions[0].head()

,American League EastAL East,W,L,PCT,GB,WCGB,L10,STRK,RS,RA,DIFF,X-W/L,HOME,AWAY,over500,Next Game
0,E-Tampa BayTB,26,15,0.634,-,-,6-4,W2,185,123,62,28-13,12-10,14-5,8-7,May 17 @ NYY
1,E-NY YankeesNYY,26,16,0.619,0.5,+3.5,8-2,W3,208,168,40,25-17,15-9,11-7,6-7,May 17 vs TB
2,E-BostonBOS,22,20,0.524,4.5,0.5,8-2,L1,224,198,26,23-19,11-8,11-12,4-6,Live vs COL
3,E-TorontoTOR,17,25,0.405,9.5,5.5,2-8,L1,151,182,-31,17-25,8-13,9-12,5-11,May 16 @ CWS
4,E-BaltimoreBAL,14,28,0.333,12.5,8.5,3-7,L2,163,242,-79,14-28,6-15,8-13,7-19,May 16 @ CLE


In [12]:
#clear standings collection before writing, as duplicate old records are not relevant
db.standings.drop()

for each_division in all_divisions:
    data = each_division.to_dict(orient='records')
    #print(data)
    db.standings.insert_many(data)
    #print('--'*12)

In [38]:
print(data)

[{'National League WestNL West': 'W-LA DodgersLAD', 'W': 28, 'L': 16, 'PCT': 0.636, 'GB': '-', 'WCGB': '-', 'L10': '7-3', 'STRK': 'W2', 'RS': 233, 'RA': 177, 'DIFF': 56, 'X-W/L': '27-17', 'HOME': '18-6', 'AWAY': '10-10', 'over500': '17-11', 'Next Game': 'Live vs SD', '_id': ObjectId('5cdcce43e52843311c721482')}, {'National League WestNL West': 'W-ArizonaARI', 'W': 24, 'L': 20, 'PCT': 0.545, 'GB': '4.0', 'WCGB': '-', 'L10': '4-6', 'STRK': 'W1', 'RS': 227, 'RA': 202, 'DIFF': 25, 'X-W/L': '24-20', 'HOME': '10-11', 'AWAY': '14-9', 'over500': '21-18', 'Next Game': 'May 17 vs SF', '_id': ObjectId('5cdcce43e52843311c721483')}, {'National League WestNL West': 'W-San DiegoSD', 'W': 22, 'L': 20, 'PCT': 0.524, 'GB': '5.0', 'WCGB': '1.0', 'L10': '4-6', 'STRK': 'L2', 'RS': 165, 'RA': 179, 'DIFF': -14, 'X-W/L': '19-23', 'HOME': '10-11', 'AWAY': '12-9', 'over500': '9-9', 'Next Game': 'Live @ LAD', '_id': ObjectId('5cdcce43e52843311c721484')}, {'National League WestNL West': 'W-ColoradoCOL', 'W': 20, 